# Experimento 4 : Análisis de arquitecturas clásicas.

En este experimento estudiaremos el comportamiento de las arquitecturas de redes neuronales más relevante a lo largo de la historia de las CNNs

## Librerías usadas.

In [ ]:
import tensorflow as tf

gpus= tf.config.experimental.list_physical_devices('GPU')
print(len(gpus))
tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import math 
from glob import glob
from matplotlib import pyplot as plt
import os
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample

## Definición de rutas

In [ ]:
#Rutas de los datos.
 
data_dir = os.path.dirname(os.path.realpath("../TFG/Datos/HAM10000_metadata.csv"))



csv_path = os.path.realpath(data_dir + "/HAM10000_metadata.csv")

#Variables globales

altura = 128
longitud = 128
clases = 7


print(data_dir)

print(csv_path)



In [ ]:
data_dir_mascara_binaria = os.path.dirname(os.path.realpath("../TFG/DatosMascaraBinaria2/HAM10000_segmentations/..."))
data_dir_mascara_superpuesta = os.path.dirname(os.path.realpath("../TFG/DatosMascaraSuperpuesta/..."))

## Creación del marco de datos.

In [ ]:
def combineData(data_dir):
    all_image_path = glob(os.path.join(data_dir, '*', '*'))
    imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}
    return imageid_path_dict

def combineData_1(data_dir):
    all_image_path = glob(os.path.join(data_dir, '*'))
    imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}
    return imageid_path_dict




#Inicializando el dataFrame

dataFrame=pd.read_csv(csv_path)

#Mezclando carpetas.

data_dict = combineData(data_dir)
data_dict_mask = combineData_1(data_dir_mascara_binaria)

def rename_keys(d, keys):
    return dict([(keys.get(k,k), v) for k, v in d.items()])

keys_values_transformer = {}
for element in data_dict_mask.keys():
    element_trans = element.replace("_segmentation","")
    keys_values_transformer[element]  = element_trans
    


data_dict_mask = rename_keys(data_dict_mask, keys_values_transformer)


data_dict_rgb_mask = combineData(data_dir_mascara_superpuesta)

# Inicializando diccionario de categorías

lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

#Añadiendo columnas al dataFrame para que sea más legible.

dataFrame['path'] = dataFrame['image_id'].map(data_dict.get)
dataFrame['mask_path'] = dataFrame['image_id'].map(data_dict_mask.get)
dataFrame['rgb_mask_path'] = dataFrame['image_id'].map(data_dict_rgb_mask.get)
dataFrame['cell_type'] = dataFrame['dx'].map(lesion_type_dict.get) 
dataFrame['cell_type_idx'] = pd.Categorical(dataFrame['cell_type']).codes

dataFrame = dataFrame.drop('dx', 1)
dataFrame = dataFrame.drop('dx_type', 1)
dataFrame = dataFrame.drop('age', 1)
dataFrame = dataFrame.drop('sex', 1)
dataFrame = dataFrame.drop('localization', 1)

dataFrame.head()

34771273B

## Se procede a crear un método que permita balancear la carga de imágenes

In [ ]:
def balanced_dataset(df):
    df_balanced = pd.DataFrame()
    #df = pd.DataFrame()
    
    for cat in df['cell_type_idx'].unique():
        temp = resample(df[df['cell_type_idx'] == cat], 
                        replace=True,     # sample with replacement
                        n_samples=2500,   # to match majority class
                        random_state=123) # reproducible results

        # Combine majority class with upsampled minority class
        df_balanced = pd.concat([df_balanced, temp])
 
    df_balanced['cell_type'].value_counts()

    return df_balanced

def load_img_data(size, df, balanced=False):
    """
        ..
        first we should normalize the image from 0-255 to 0-1
    """
    
    img_h, img_w = size, size
    imgs = []
    
    if balanced:
        df = balanced_dataset(df)
    
    image_paths = list(df['path'])

    for i in tqdm(range(len(image_paths))):
        img = cv2.imread(image_paths[i])
        img = cv2.resize(img, (img_h, img_w))
        img = img.astype(np.float32) / 255.
        #img = np.asarray(Image.open(image_paths[i]).resize((size,size)))
        imgs.append(img)

    imgs = np.stack(imgs, axis=0)
    print(imgs.shape)

    #imgs = imgs.astype(np.float32) / 255.
    
    return imgs, df['cell_type_idx'].values

def load_img_data_segmentation(size, df, balanced=False):
    """
        ..
        first we should normalize the image from 0-255 to 0-1
    """
    
    img_h, img_w = size, size
    imgs = []
    imgs_segmented = []
    
    if balanced:
        df = balanced_dataset(df)
    
    image_paths = list(df['path'])
    image_paths_1 = list(df['mask_path'])

    imgs = preproces_image(image_paths,img_h, img_w)
    imgs_seg = preproces_image(image_paths_1,img_h, img_w)

    #imgs = imgs.astype(np.float32) / 255.
    
    return imgs,imgs_seg, df['cell_type_idx'].values


def preproces_image(image_paths,img_h, img_w):
    imgs = []
    for i in tqdm(range(len(image_paths))):
        img = cv2.imread(image_paths[i])
        img = cv2.resize(img, (img_h, img_w))
        img = img.astype(np.float32) / 255.
        #img = np.asarray(Image.open(image_paths[i]).resize((size,size)))
        imgs.append(img)

    imgs = np.stack(imgs, axis=0)
    print(imgs.shape)
    return imgs
    

In [ ]:
imgs, seg_imgs, values = load_img_data_segmentation(128, dataFrame, balanced=True)

In [ ]:
def load_general_data(imgs, seg_imgs, values):
    
    x_train_img,x_test_img,y_train_seg,y_test_seg,y_train_clas,y_test_clas = train_test_split(imgs, seg_imgs,values, test_size=0.40,random_state=123)

       
    train_data = [ x_train_img , y_train_seg,y_train_clas]
    test_data = [ x_test_img , y_test_seg, y_test_clas ]
    
    x_test_img,x_val_img,y_test_seg,y_val_seg,y_test_clas,y_val_clas = train_test_split(test_data[0], test_data[1],test_data[2], test_size=0.60,random_state=123)

    
    
    train_data = [x_train_img , y_train_seg,y_train_clas]
    val_data   = [x_val_img , y_val_seg, y_val_clas]
    test_data  = [x_test_img , y_test_seg,y_test_clas]
    
    print("Work DONE")
    
    return train_data,val_data,test_data

train_data,val_data,test_data = load_general_data(imgs, seg_imgs, values)

## Cargamos los datos y creamos los casos a experimentar.

In [ ]:
def load_general_data():
    
    imgs, target = load_img_data(altura, dataFrame, balanced=True)
    
    x_train, x_transferLearning, y_train, y_transferLearning = train_test_split(imgs, target, test_size=0.60,random_state=123)
       
    source_data = [ x_transferLearning , y_transferLearning ]
    target_data = [ x_train , y_train ]
    
    x_train,x_test,y_train,y_test = train_test_split(target_data[0], target_data[1], test_size=0.70,random_state=123)
    
    train_data = [x_train,y_train]
    test_data = [x_test,y_test]
    
    return source_data,train_data,test_data


def get_data_for_ex(source_data,train_data,test_data):
    
    x_train = source_data[0]
    y_train = source_data[1]
    
    x_retrain = train_data[0]
    y_retrain = train_data[1]
    
    percent = math.floor(len(test_data[0])/100*30)
       
    x_validation = test_data[0][0:percent]
    y_validation = test_data[1][0:percent]
    
    
    x_test = test_data[0][percent:-1]
    y_test = test_data[1][percent:-1]
    
    return x_train,x_retrain,x_test,x_validation,y_train,y_retrain,y_test,y_validation


###############################################################################################################
# Definimos 7 experimentos cada uno con un optimizador distingo y definimos el número de iteraciones          #
###############################################################################################################

ITERATIONS_PER_EXP = 5
BATCH_SIZE = 16
EPOCHS = 20
LEARNING_RATE=0.0001


def set_hiper_to_exp(BATCH_SIZE,EPOCHS,LEARNING_RATE):
    opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE,amsgrad=True)  
    return BATCH_SIZE,EPOCHS,opt

BATCH_SIZE,EPOCHS,opt = set_hiper_to_exp(BATCH_SIZE,EPOCHS,LEARNING_RATE)

In [ ]:
def clasifier(x,out_name):
    #x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(64)(x)
    x = tf.keras.layers.PReLU()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(64)(x)
    x = tf.keras.layers.PReLU()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    
    x = tf.keras.layers.Dense(7,activation='softmax',name=out_name)(x)
    
    return x
    
    x = conv_block(image,32,kernel_size=(3,3),with_bn=with_bn)
    x = conv_block(x,32,kernel_size=(3,3),with_bn=with_bn)
    x = tf.keras.layers.MaxPooling2D(2,2)(x)
    
    
    x = conv_block(x,64,kernel_size=(3,3),with_bn=with_bn)
    x = tf.keras.layers.MaxPooling2D(2,2)(x)
    
def encoder(image):
    
    conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(image)
    #conv1 = tf.keras.layers.PReLU()(conv1)
    #conv1 = tf.keras.layers.BatchNormalization()(conv1)
    conv1 = tf.keras.layers.Dropout(0.2)(conv1)
    
    conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    #conv1  = tf.keras.layers.PReLU()(conv1)
    #conv1 = tf.keras.layers.BatchNormalization()(conv1)
    conv1 = tf.keras.layers.Dropout(0.2)(conv1)
    
    pool1 = tf.keras.layers.MaxPooling2D((2, 2))(conv1)
    
    conv2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    #conv2 = tf.keras.layers.PReLU()(conv2)
    #conv2 = tf.keras.layers.BatchNormalization()(conv2)
    #conv1 = tf.keras.layers.Dropout(0.2)(conv1)
    pool2 = tf.keras.layers.MaxPooling2D((2, 2))(conv2)
    
    return pool2,conv2,conv1

def decoder(pool2,conv2,conv1):
    
    conv3 = tf.keras.layers.Conv2D(128, (3, 3), activation='elu', padding='same')(pool2)
    conv3 = tf.keras.layers.Dropout(0.2)(conv3)
    #conv3 = tf.keras.layers.BatchNormalization()(conv3)    
    conv3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = tf.keras.layers.Dropout(0.2)(conv3)

    up1 = tf.keras.layers.concatenate([tf.keras.layers.UpSampling2D((2, 2))(conv3), conv2], axis=-1)
    conv4 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
    conv4 = tf.keras.layers.Dropout(0.2)(conv4)
    #conv4 = tf.keras.layers.BatchNormalization()(conv4)
    conv4 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

    up2 = tf.keras.layers.concatenate([tf.keras.layers.UpSampling2D((2, 2))(conv4), conv1], axis=-1)
    conv5 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
    conv5 = tf.keras.layers.Dropout(0.2)(conv5)
    #conv5 = tf.keras.layers.BatchNormalization()(conv5)
    conv5 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)
    
    out = tf.keras.layers.Conv2D( 1, (1, 1) , padding='same',name="decoder_output")(conv5)
    
    return out

def clasifier_v2(x,out_name):
    #x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(256)(x)
    x = tf.keras.layers.PReLU()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(256)(x)
    x = tf.keras.layers.PReLU()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    
    x = tf.keras.layers.Dense(7,activation='softmax',name=out_name)(x)
    
    return x
    
def encoder_v2(image):
    
    conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(image)
    #conv1 = tf.keras.layers.PReLU()(conv1)
    #conv1 = tf.keras.layers.BatchNormalization()(conv1)
    conv1 = tf.keras.layers.Dropout(0.2)(conv1)
    
    conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    #conv1  = tf.keras.layers.PReLU()(conv1)
    #conv1 = tf.keras.layers.BatchNormalization()(conv1)
    conv1 = tf.keras.layers.Dropout(0.2)(conv1)
    
    pool1 = tf.keras.layers.MaxPooling2D((2, 2))(conv1)

    conv2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    #conv2 = tf.keras.layers.PReLU()(conv2)
    #conv2 = tf.keras.layers.BatchNormalization()(conv2)
    conv2 = tf.keras.layers.Dropout(0.2)(conv2)
    pool2 = tf.keras.layers.MaxPooling2D((2, 2))(conv2)
    
    conv3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = tf.keras.layers.Dropout(0.2)(conv3)
    #conv3 = tf.keras.layers.BatchNormalization()(conv3)
    conv3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    
    return conv3,conv2,conv1

def decoder_v2(conv3,conv2,conv1):
    
    

    up1 = tf.keras.layers.concatenate([tf.keras.layers.UpSampling2D((2, 2))(conv3), conv2], axis=-1)
    conv4 = tf.keras.layers.Conv2D(64, (3, 3), activation='elu', padding='same')(up1)
    conv4 = tf.keras.layers.Dropout(0.2)(conv4)
    conv4 = tf.keras.layers.BatchNormalization()(conv4)
    conv4 = tf.keras.layers.Conv2D(64, (3, 3), activation='elu', padding='same')(conv4)

    up2 = tf.keras.layers.concatenate([tf.keras.layers.UpSampling2D((2, 2))(conv4), conv1], axis=-1)
    conv5 = tf.keras.layers.Conv2D(32, (3, 3), activation='elu', padding='same')(up2)
    conv5 = tf.keras.layers.Dropout(0.2)(conv5)
    #conv5 = tf.keras.layers.BatchNormalization()(conv5)
    conv5 = tf.keras.layers.Conv2D(32, (3, 3), activation='elu', padding='same')(conv5)
    
    out = tf.keras.layers.Conv2D( 1, (1, 1) , padding='same',name="decoder_output")(conv5)
    
    return out

In [ ]:
def enc_dec_v1(input_shape=(128, 128, 3)):
    image = tf.keras.layers.Input(shape=input_shape)
    conv3,conv2,conv1 = encoder(image)
    out1 = decoder(conv3,conv2,conv1)
    out = clasifier(conv3,"softmax_output")
    model = tf.keras.models.Model(image,outputs=[out,out1])
    
    model.summary()
    return model

def enc_dec_v2(input_shape=(128, 128, 3)):
    image = tf.keras.layers.Input(shape=input_shape)
    conv3,conv2,conv1 = encoder_v2(image)
    out1,conv4,conv5 = decoder_v2(conv3,conv2,conv1)
    out,conv4,conv5 = clasifier_v2(conv3,conv4,conv5,"softmax_output")
    model = tf.keras.models.Model(image,outputs=[out,out1])
    
    model.summary()
    return model

def clasiffier(input_shape=(128, 128, 3)):
    image = tf.keras.layers.Input(shape=input_shape)
    conv3,conv2,conv1 = encoder(image)
    out = clasifier(conv3,"softmax_output")
    
    model = tf.keras.models.Model(image,outputs=[out])
    
    model.summary()
    return model

model = clasiffier()

In [ ]:
def build(nn):
    model = tf.keras.Sequential()
    model.add(nn)
    model.add(tf.keras.layers.Flatten())
    
    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.PReLU())

    model.add(tf.keras.layers.Dense(7,activation='softmax'))

    print(model.summary())

    return model

def conv_block(x,filters,kernel_size = (3,3),strides = (1,1),with_bn=False):
    x = tf.keras.layers.Conv2D(filters=filters,kernel_size=kernel_size,strides=strides)(x)
    x  = tf.keras.layers.PReLU()(x)
    if with_bn: 
        x = tf.keras.layers.BatchNormalization()(x)
    return x

def full_build_cnn_soco_v4(clases = 7,with_bn = True, with_dropout=False,stn=0,input_shape=(128, 128, 3)):
    image = tf.keras.layers.Input(shape=input_shape)
    
    x = conv_block(image,32,kernel_size=(3,3),with_bn=with_bn)
    x = conv_block(x,32,kernel_size=(3,3),with_bn=with_bn)
    x = tf.keras.layers.MaxPooling2D(2,2)(x)
    
    
    x = conv_block(x,64,kernel_size=(3,3),with_bn=with_bn)
    x = tf.keras.layers.MaxPooling2D(2,2)(x)
    
      
    model = tf.keras.models.Model(inputs=image, outputs=x)
    model.summary()
    return model

In [ ]:
nn = full_build_cnn_soco_v4()
nn = build(nn)

In [ ]:
model = clasiffier()

earlyStopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10,
                                                         mode = 'min')

#cb_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, verbose=1,
                                                     #patience=4, min_delta=1e-3, 
                                                     #cooldown=0, min_lr=0.000001)

opt = tf.keras.optimizers.Adam(learning_rate=0.001,amsgrad=True)        

model.compile(optimizer=opt,
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])


history = model.fit(x_train,y_train_clas,
                     validation_data= (x_val_img,y_val_clas),
                     epochs=60,
                     callbacks = [earlyStopping,cb_reduce_lr],
                     batch_size=16,verbose=2)

In [ ]:
x_train = train_data[0]
y_train_seg = train_data[1]
y_train_clas = train_data[2]

x_val_img = val_data[0]
y_val_seg = val_data[1]
y_val_clas = val_data[2]

In [ ]:
print(x_train.shape)
print(y_train_seg.shape)
print(y_train_clas.shape)
print(x_val_img.shape)
print(y_val_seg.shape)
print(y_val_clas.shape)

In [ ]:
def train_model(iteraciones):
    evaluations = []
    for i in range(iteraciones):
        
        model = enc_dec_v2()
        
        checkpoint ="../TFG/Modelos/prop_500_light_"+str(i)+".h5"
        
        cpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint, monitor="val_loss",
                                                    mode="min", save_best_only=True, verbose=0)
        
        earlyStopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5,
                                                         mode = 'min')
        
        opt = tf.keras.optimizers.Adam(learning_rate=0.001,amsgrad=True)
        
        losses = {
            "decoder_output": "binary_crossentropy",
            "softmax_output": "sparse_categorical_crossentropy",
        }

        lossWeights = {
            "decoder_output": 1.0,
            "softmax_output": 1.0
        }
    
        cb_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, verbose=1,
                                                     patience=2, min_delta=1e-3, 
                                                     cooldown=0, min_lr=0.00001)
        
        model.compile(optimizer=opt,
              loss=losses,
              loss_weights=lossWeights,
              metrics=['accuracy'])


        history = model.fit(x_train,[y_train_clas,y_train_seg],
                     validation_data= (x_val_img,[y_val_clas,y_val_seg]),
                     epochs=60,
                     callbacks = [cpoint,earlyStopping,cb_reduce_lr],
                     batch_size=16,verbose=2)
        
        evaluation = history.model.evaluate(test_data[0],[test_data[2],test_data[1]])
        evaluations.append(evaluation)
    
    return evaluations
    
evaluations = train_model(10)

In [ ]:
history.model.evaluate(test_data[0],test_data[2])

In [ ]:
import statistics as stats

classifiers_acc = []
decoders_acc = []

for [gloss,closs,dloss,cacc,dacc] in evaluations:
    print(str(cacc)+" "+str(dacc))
    classifiers_acc.append(cacc)
    decoders_acc.append(dacc)

print("-------------------------")
print(stats.mean(classifiers_acc))
print(stats.mean(decoders_acc))
print("-------------------------")
    

In [ ]:
model = tf.keras.models.load_model("../TFG/Modelos/prop"+str(0)+".h5")

model.evaluate(test_data[0],[test_data[2],test_data[1]])

In [ ]:
i=343
  

    
def print_prediction(i):
    pixels = np.array(x_test[i])
    image = cv2.cvtColor(pixels, cv2.COLOR_BGR2RGB) 

    gt = np.array(y_test_seg[i])

    i,m = model.model.predict(x_test[i][None,:,:,:])
    prediction = m[0,:,:,:]>=0.5

    
    display_list = [image,gt,prediction]
    
    title= ["Input Image","Ground Truth","Predicted Image"]

    for i in range(3):
        plt.subplot(1,3,i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis("off")
    plt.show()  
    

for i in range(0,10):
    print_prediction(i)


In [ ]:
plt.imshow(m[0,:,:,:]>= 0.5, cmap='gray')

In [ ]:
plt.imshow(y_test_seg[3], cmap='gray')

In [ ]:
pixels = np.array(x_test[3])
image = cv2.cvtColor(pixels, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()

In [ ]:
def combineData(data_dir):
    all_image_path = glob(os.path.join(data_dir, '*', '*'))
    imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}
    return imageid_path_dict

In [ ]:
preds_test_thresh = (test_preds >= 0.5).astype(np.uint8)
test_img = preds_test_thresh[5, :, :, 0]
plt.imshow(test_img, cmap='gray')

In [ ]:
test_preds = model.model.predict(x_test[0][None,:,:,:])

In [ ]:
number_img = 8
pixels = np.array(x_test[0])
image = cv2.cvtColor(pixels, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()



preds_test_thresh = (test_preds[0] >= 0.4).astype(np.uint8)
pixels = np.array(preds_test_thresh)
plt.imshow(pixels, cmap='gray')
plt.show()

In [ ]:
pixels = np.array(y_test[0])
plt.imshow(pixels, cmap='gray')
plt.show()

pixels = np.array(test_preds[0])
plt.imshow(pixels, cmap='gray')
plt.show()